# Forest Cover Type Prediction: Data Cleaning and Exploratory Analysis

This notebook performs data cleaning and exploratory data analysis (EDA) on the Forest Cover dataset. The primary goal is to preprocess the data, handle missing values, and visualize key relationships between features and the target variable, Forest_Cover

## Step 1: Data Loading and Initial Exploration

First, load the necessary libraries and the dataset and define the data types for each column to ensure they are read correctly.

In [ ]:
import numpy as np
import pandas as pd

# Define the data types for each column to ensure correct loading
# 'key_list' contains the initial numerical columns and the target variable.
key_list = [
    'Elevation',
    'Aspect',
    'Slope',
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points',
    'Forest_Cover'
]
# 'value_list' defines the corresponding data types.
value_list = [
    'float64',
    'float64',
    'float64',
    'float64',
    'float64',
    'float64',
    'float64',
    'float64',
    'float64',
    'float64',
    'object' # Forest_Cover is an object since it's a categorical target
]

dtype_dict = dict(zip(key_list, value_list))
# Add the 40 Soil_Type columns and 4 Wilderness_Area columns, which are one-hot encoded and should be integers.
for i in range(1, 41):
    dtype_dict[f'Soil_Type{i}'] = 'Int64'

dtype_dict.update({
    'Neota': 'Int64',
    'Rawah': 'Int64',
    'Comanche Peak': 'Int64',
    'Cache la Poudre': 'Int64'
})

# Load the data from the CSV file using the predefined data types.
forest_data = pd.read_csv('./dataset/forest_cover.csv',
                          dtype=dtype_dict,
                          on_bad_lines='skip',
                          engine='python')
# Remove the unnamed index column that often appears when saving dataframes to CSV.
forest_data.drop('Unnamed: 0', axis=1, inplace=True)
# forest_data.dtypes
forest_data.shape
# forest_data.head()

Find out the type of different columns

In [ ]:
columns_data_types = forest_data.dtypes.reset_index()
columns_data_types

Calculate the missing value rate for each column.

In [ ]:
columns_names = forest_data.columns.values
columns_missing_rate = forest_data.isna().mean().round(5).reset_index()
columns_missing_rate

Merge the data types and missing value rates into a single dataframe for easy viewing and export.


In [ ]:
data_type_missing_value = pd.merge(columns_data_types, columns_missing_rate, on='index')

data_type_missing_value.rename(columns={'index': 'column', '0_x': 'type', '0_y': 'missing_rate'}, inplace=True)
data_type_missing_value.to_csv("missing_value.csv", index=False)

## Step 2: Visualizing Missing Values and unique value of columns


Create the chart to show missing value rate

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Create a bar chart
plt.figure(figsize=(12, 6))
plt.bar(data_type_missing_value['column'], data_type_missing_value['missing_rate'], color='skyblue')
# Format y-axis to display percentages.
plt.gca().yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1.0))

plt.xlabel('Column', fontsize=12)
plt.ylabel('Missing Rate', fontsize=12)
plt.title('Missing Rate of Different Columns', fontsize=14)

# Rotate x-axis labels
plt.xticks(rotation=45, ha='right')

# Ensure the layout is tight to prevent labels from being cut off
plt.tight_layout()

plt.savefig('missing_rate_bar_chart.png')

Get the unique values of each attribute, and make an example showing a maximum of 3 values.

In [ ]:
count_unique_attribute = []
for name in columns_names:
    count = pd.unique(forest_data[name])

    # Filter out missing values (like NaN) for the example string
    filtered_examples = [item for item in count if pd.notna(item)]

    row_data = {
        'Attribute': name,
        'Count of unique value': len(count),
        'Example': ", ".join(pd.Series(filtered_examples[:3]).astype(str)),
    }
    count_unique_attribute.append(row_data)
# export the data to csv file
pd.DataFrame(count_unique_attribute).to_csv('count_unique_value.csv', index=False)
count_unique_attribute

## Step 3: Data Cleaning

This step focuses on cleaning the data by handling invalid entries and missing values.

Replace rows with incorrect data in Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,	Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points like negative and out of range. And remove incorrect soil_type rows without certain type

In [ ]:
# Create a copy of the original dataframe to perform cleaning.
forest_data_test1 = forest_data.copy()

### Handling invalid soil type columns


Filter out invalid soil type rows because it's hard to estimate the real type if there is no exact value. Filling in any type with 1 is not reliable.

In [ ]:
# filter out wrong soil-type by calculate the sum of them, if the result is 0, remove the row
# because soil type columns use one-hot encoding method, which 1 indicates existing and 0 means no. If a whole row has no 1, it's impossible to decide which one is right. As a result, remove these rows to improve the accuracy.
soil_type_columns = [f'Soil_Type{i}' for i in range(1, 41)]
# fill the nan with 0 to calculate the sum of soil type
forest_data_test1[soil_type_columns] = forest_data_test1[soil_type_columns].fillna(0)
forest_data_test1.head()

In [ ]:
# calculate the sum of different soil type
forest_data_test1['Soil_Type_Sum'] = forest_data_test1[soil_type_columns].sum(axis=1)
forest_data_test1.head()

In [ ]:
# Filter out the rows where the sum of the 40 Soil_Type columns is 0, which means this row is no valid.
filtered_df = forest_data_test1[forest_data_test1['Soil_Type_Sum'] == 1].copy()
# remove the sum column
filtered_df = filtered_df.drop(columns=['Soil_Type_Sum'])
filtered_df.head()

### Handling Out-of-Range Numerical Values


In [ ]:
# Define numerical columns to be checked for valid ranges.
numerical_cols = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
                  'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
                  'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                  'Horizontal_Distance_To_Fire_Points']

# Define valid ranges
ranges = {
    'Elevation': (500, 5000), 'Aspect': (0, 360), 'Slope': (0, 90),
    'Horizontal_Distance_To_Hydrology': (0, np.inf),
    'Horizontal_Distance_To_Roadways': (0, np.inf),
    'Horizontal_Distance_To_Fire_Points': (0, np.inf),
    'Hillshade_9am': (0, 255), 'Hillshade_Noon': (0, 255), 'Hillshade_3pm': (0, 255)
}

# Iterate through the defined ranges and replace out-of-range values with NaN.
for col, (min_val, max_val) in ranges.items():
    if col in filtered_df.columns:
        filtered_df.loc[~filtered_df[col].between(min_val, max_val), col] = np.nan

filtered_df.head()

### Imputing Missing Values

Three common imputation methods, median, mean, and mode to fill in the remaining missing numerical values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set a style for the plot for better aesthetics.
sns.set_style("whitegrid")

In [ ]:
# Define a helper function to fill in missing values using a specified method.
def fillInMissing(data, fn):
    for key_name in key_list:
        if pd.api.types.is_numeric_dtype(data[key_name]):
            if fn == 'mode':
                # Explicitly get the first mode
                fill_value = data[key_name].mode()[0]
            else:
                fill_value = data[key_name].agg(fn)
            print(f'{key_name}  {fn}_value:{fill_value}')
            data[key_name] = data[key_name].fillna(fill_value)
    print('\n')
    return data

In [ ]:
# Create copies of the filtered dataframe for each imputation method.
filtered_df_test_median = filtered_df.copy()
filtered_df_test_mean = filtered_df.copy()
filtered_df_test_mode = filtered_df.copy()

fill_with_median_path = './plots/fill_with_median_'
fill_with_mean_path = './plots/fill_with_mean_'
fill_with_mode_path = './plots/fill_with_mode_'

# The scenarios list as defined it
scenarios = [
    {'df': filtered_df_test_median, 'method': 'median', 'path': fill_with_median_path},
    {'df': filtered_df_test_mean, 'method': 'mean', 'path': fill_with_mean_path},
    {'df': filtered_df_test_mode, 'method': 'mode', 'path': fill_with_mode_path}
]

# fill in
for i, scenario in enumerate(scenarios):
    fillInMissing(scenario['df'], scenario['method'])

## Step 4: Exploratory Data Analysis (EDA) and Visualization


 Explore the relationships between different features and the target variable, Forest_Cover. Select numerical features like Elevation and Slope, Categorical Features like Wilderness_Area and soil types


### Analyzing Numerical Features

In [ ]:
# Calculate and print the correlation between Elevation and Slope.
for i, scenario in enumerate(scenarios):
    correlation = scenario['df'][['Elevation', 'Slope']].corr()
    print(f'Correlation between Elevation and Slope {scenario['method']} value: ')
    print(correlation)
    # Add the calculated correlation to the dictionary in the original list
    scenarios[i]['correlation'] = correlation

In [ ]:
# Create a heatmap of the correlation matrix.
for i, scenario in enumerate(scenarios):
    # Get the stored values from the scenario dictionary
    method = scenario['method']
    correlation = scenario['correlation']
    path = scenario['path']

    # Create a heatmap of the correlation
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title(f'Correlation Matrix of Elevation and Slope ({method} value)')
    plt.savefig(path + 'Elevation_Slope_correlation_heatmap.png')

In [ ]:
# Create a scatter plot to visualize the relationship between Elevation and Slope.
for i, scenario in enumerate(scenarios):
    path = scenario['path']
    df = scenario['df']
    method = scenario['method']
    # Create a scatter plot
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='Elevation', y='Slope', data=df)
    plt.title(f'Scatter Plot of Elevation vs. Slope ({method} value)')
    plt.xlabel('Elevation')
    plt.ylabel('Slope')
    plt.grid(True)
    plt.savefig(path + 'Elevation_Slope_scatter_plot.png')

### Relationship with the Target Variable

Use box plots and bar plots to understand how `Elevation` and `Slope` relate to the `Forest_Cover` types.

In [ ]:
# Box plot for Elevation and Forest Cover.
for i, scenario in enumerate(scenarios):
    path = scenario['path']
    df = scenario['df']
    method = scenario['method']
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='Forest_Cover', y='Elevation', data=df)
    # Add a title and labels for clarity.
    plt.title(f'Distribution of Forest Cover Type by Elevation ({method} value)', fontsize=16)
    plt.xlabel('Forest Cover Type', fontsize=12)
    plt.ylabel('Elevation', fontsize=12)

    # Rotate x-axis labels to prevent overlap, making them easier to read.
    plt.xticks(rotation=45, ha='right')
    # Adjust layout to make sure labels are not cut off.
    plt.tight_layout()

    # Display the plot.
    plt.savefig(path + 'Elevation_Forest_Cover_boxplot.png')
    plt.show()

In [ ]:
# Box plot for Slope and Forest Cover.
for i, scenario in enumerate(scenarios):
    path = scenario['path']
    df = scenario['df']
    method = scenario['method']

    plt.figure(figsize=(12, 8))
    sns.boxplot(x='Forest_Cover', y='Slope', data=df)
    # Add a title and labels for clarity.
    plt.title(f'Distribution of Forest Cover Type by Slope ({method} value)', fontsize=16)
    plt.xlabel('Forest Cover Type', fontsize=12)
    plt.ylabel('Slope', fontsize=12)

    # Rotate x-axis labels to prevent overlap, making them easier to read.
    plt.xticks(rotation=45, ha='right')
    # Adjust layout to make sure labels are not cut off.
    plt.tight_layout()

    # Display the plot.
    plt.savefig(path + 'Slope_Forest_Cover_boxplot.png')
    plt.show()

In [ ]:
# Bar plot for Average Elevation by Forest Cover Type.
for i, scenario in enumerate(scenarios):
    path = scenario['path']
    df = scenario['df']
    method = scenario['method']

    # Calculate the average elevation for each forest cover type.
    average_elevation = df.groupby('Forest_Cover')['Elevation'].mean().reset_index()

    # Create the bar plot showing the average elevation for each forest cover type.
    # The hue parameter is now set to the same variable as x to avoid a deprecation warning.
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Forest_Cover', y='Elevation', data=average_elevation, hue='Forest_Cover', palette='viridis',
                legend=False)

    # Add a title and labels for clarity.
    plt.title(f'Average Elevation by Forest Cover Type ({method} value)', fontsize=16)
    plt.xlabel('Forest Cover Type', fontsize=12)
    plt.ylabel('Average Elevation', fontsize=12)

    # Rotate x-axis labels to prevent overlap, making them easier to read.
    plt.xticks(rotation=45, ha='right')

    # Adjust layout to make sure labels are not cut off.
    plt.tight_layout()

    # Display the plot.
    plt.savefig(path + 'Average_Elevation_Forest_Cover_boxplot.png')
    plt.show()


In [ ]:
# Bar plot for Average Slope by Forest Cover Type.
for i, scenario in enumerate(scenarios):
    path = scenario['path']
    df = scenario['df']
    method = scenario['method']

    # Calculate the average elevation for each forest cover type.
    average_elevation = df.groupby('Forest_Cover')['Slope'].mean().reset_index()

    # Create the bar plot showing the average elevation for each forest cover type.
    # The hue parameter is now set to the same variable as x to avoid a deprecation warning.
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Forest_Cover', y='Slope', data=average_elevation, hue='Forest_Cover', palette='viridis',
                legend=False)

    # Add a title and labels for clarity.
    plt.title(f'Average Slope by Forest Cover Type ({method} value)', fontsize=16)
    plt.xlabel('Forest Cover Type', fontsize=12)
    plt.ylabel('Average Slope', fontsize=12)

    # Rotate x-axis labels to prevent overlap, making them easier to read.
    plt.xticks(rotation=45, ha='right')

    # Adjust layout to make sure labels are not cut off.
    plt.tight_layout()

    # Display the plot.
    plt.savefig(path + 'Average_Slope_Forest_Cover_boxplot.png')
    plt.show()


### Analyzing Categorical Features

This section explores the relationship between the categorical features (Area and Soil_Type) and the Forest_Cover target variable. Since these are one-hot encoded, we will visualize their proportions.

In [ ]:
# Identify Wilderness Area columns
wilderness_cols = ['Neota', 'Rawah', 'Comanche Peak', 'Cache la Poudre']

# Melt the DataFrame to one to analyze the relationship between Forest Cover and Wilderness Area.
df_wild_melted = filtered_df.melt(id_vars='Forest_Cover', value_vars=wilderness_cols, var_name='Wilderness_Area',
                                  value_name='Is_Present') # The 'Is_Present' column will contain 1s for the rows where a specific wilderness area is present.
df_wild_melted

In [ ]:
# Filter for rows where the wilderness area is present (value is 1)
df_wild_filtered = df_wild_melted[df_wild_melted['Is_Present'] == 1]
# Calculate the counts for each combination of Forest_Cover and Wilderness_Area
wild_counts = df_wild_filtered.groupby(['Forest_Cover', 'Wilderness_Area']).size().unstack(fill_value=0)
wild_counts

In [ ]:
# Calculate the total count for each Forest_Cover type
forest_cover_totals = wild_counts.sum(axis=1)
# Calculate the normalized proportions
wild_proportions = wild_counts.div(forest_cover_totals, axis=0)
wild_proportions

In [ ]:
# # Create a bar chart to show the proportions
fig, ax = plt.subplots(figsize=(12, 8))
wild_proportions.plot(kind='bar', stacked=True, ax=ax)
ax.set_title('Normalized Proportions of Wilderness Areas per Forest Cover Type')
ax.set_xlabel('Forest Cover Type')
ax.set_ylabel('Proportion')
ax.tick_params(axis='x', rotation=45)
plt.legend(title='Wilderness Area', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('wilderness_area_proportions.png')

In [ ]:
# Create a heatmap to provide a clearer visualization of the proportions.
plt.figure(figsize=(12, 8))
sns.heatmap(wild_proportions, cmap='YlGnBu', annot=True, fmt=".2f", linewidths=.5)
plt.title('Normalized Proportions of Wilderness Areas per Forest Cover Type')
plt.xlabel('Wilderness Area')
plt.ylabel('Forest Cover Type')
plt.tight_layout()
plt.savefig('wilderness_area_heatmap.png')

Soil Types

In [ ]:
# Define the list of Soil_Type columns.
soil_cols = [f'Soil_Type{i}' for i in range(1, 41)]

# Melt the DataFrame to long format for Soil Types
df_soil_melted = filtered_df.melt(id_vars='Forest_Cover', value_vars=soil_cols, var_name='Soil_Type',
                                  value_name='Is_Present')
df_soil_melted

In [ ]:
# Filter for rows where the soil type is present (value is 1)
df_soil_filtered = df_soil_melted[df_soil_melted['Is_Present'] == 1]
df_soil_filtered

In [ ]:
# Calculate the counts for each combination of Forest_Cover and Soil_Type
soil_counts = df_soil_filtered.groupby(['Forest_Cover', 'Soil_Type']).size().unstack(fill_value=0)
soil_counts = soil_counts.reindex(columns=soil_cols, fill_value=0)
soil_counts

In [ ]:
# Calculate the total count for each Forest_Cover type
forest_cover_totals = soil_counts.sum(axis=1)
# Calculate the normalized proportions
soil_proportions = soil_counts.div(forest_cover_totals, axis=0)

# Extract the numeric part from the column names (e.g., 'Soil_Type1' -> 1)
numeric_labels = [int(col.replace('Soil_Type', '')) for col in soil_proportions.columns]

# Set the column names to be the numeric labels
soil_proportions.columns = numeric_labels

# Create a heatmap
plt.figure(figsize=(18, 10))
sns.heatmap(soil_proportions, cmap='YlGnBu', annot=True, fmt=".2f", linewidths=.5)
plt.title('Normalized Proportions of Soil Types per Forest Cover Type')
plt.xlabel('Soil Type')
plt.ylabel('Forest Cover Type')
plt.tight_layout()
plt.savefig('soil_type_heatmap.png')

## Export the median value one to csv file as final result

In [ ]:
for i, scenario in enumerate(scenarios):
    if scenario['method'] == "median":
        scenario['df'].to_csv(f'./dataset_cleaned_filled/forest_cover_filled_median.csv')